In [1]:
import numpy as np
import cv2
import os

In [2]:
from Utils.DbTools import *

Using TensorFlow backend.


In [3]:
images = load_files("./datasets/Lara3D_UrbanSeq1_JPG/",select="jpg")

In [4]:
# comment bear
#       ∩＿＿＿∩
#      |ノ      ヽ
#     /   ●    ● | クマ──！！
#    |     (_●_) ミ
#   彡､     |∪|  ､｀＼
#  / ＿＿    ヽノ /´>   )
# (＿＿＿）     /  (_／
#   |        /
#   |   ／＼  ＼
#   | /     )   )
#   ∪     （   ＼
#            ＼＿)

import numpy as np
import os
import six.moves.urllib as urllib # 這是用來讓不同version的python能夠好好使用的。
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion # 用來將version的版號分離的工具, 反推回去會有一個工具是幫你寫好版號的
from collections import defaultdict # Containner 如 dict, list, set, tuple 實現的地方
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image # 用來Open Image的方法

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

from utils import label_map_util

from utils import visualization_utils as vis_util

# if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
#   raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')

In [18]:
# What model to download.
MODEL_NAME = 'faster_rcnn_resnet101_kitti_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

In [19]:
if not os.path.isfile(MODEL_FILE):
    print("File not found! Download from net")
    opener = urllib.request.URLopener()
    opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    tar_file = tarfile.open(MODEL_FILE)
    for file in tar_file.getmembers():
      file_name = os.path.basename(file.name)
      if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())
else:
    print("Already exist file :"+ MODEL_FILE+'/frozen_inference_graph.pb')

Already exist file :faster_rcnn_resnet101_kitti_2018_01_28.tar.gz/frozen_inference_graph.pb


In [20]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [21]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [22]:
PATH_TO_LABELS = "data/kitti_label_map.pbtxt"
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [23]:
print(category_index)

{1: {'id': 1, 'name': 'car'}, 2: {'id': 2, 'name': 'pedestrian'}}


In [24]:
def run_inference_for_single_image(image, graph, sess):
        # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
      return output_dict

In [ ]:
cap = cv2.VideoCapture('test.avi')
with detection_graph.as_default():
    with tf.Session() as sess:
        while(cap.isOpened()):
              ret, image_np = cap.read()
              image_np_expanded = np.expand_dims(image_np, axis=0)
          # Actual detection.
              output_dict = run_inference_for_single_image(image_np, detection_graph,sess)
              # Visualization of the results of a detection.
              vis_util.visualize_boxes_and_labels_on_image_array(
                  image_np,
                  output_dict['detection_boxes'],
                  output_dict['detection_classes'],
                  output_dict['detection_scores'],
                  category_index,
                  instance_masks=output_dict.get('detection_masks'),
                  use_normalized_coordinates=True,
                  line_thickness=8)
              cv2.imshow('object detection',cv2.resize(image_np,(800,600)))
              if cv2.waitKey(25) & 0xff == ord('q'):
                  cv2.destroyAllWindows()
                  break
        cap.release()
        cv2.destroyAllWindows()